<a href="https://colab.research.google.com/github/Ahmed-M2020/Ahmed-M2020/blob/main/part1_hff.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
from shutil import copyfile
import pandas as pd
from pathlib import Path
import os
import numpy as np
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
file_path = '/content/drive/MyDrive/zerop/0_Auswertung_231115.csv'
df = pd.read_csv(file_path, sep='\t', encoding='ISO-8859-1')

In [3]:
folder_path = Path('/content/drive/MyDrive/zerop/FTI Dataset 2023')
all_image_paths = list(folder_path.rglob('*.png'))
base_names = [os.path.basename(path) for path in all_image_paths]
path_mapping = dict(zip(base_names, all_image_paths))
columns = ['File','Feature','Label']
df2 = df.loc[:, columns]
df2['absolute_path'] = df2['File'].map(path_mapping)
# df2['Label'] = df2['Label'].replace({'good': 0, 'bad': 1})
df2.head()

,File,Feature,Label,absolute_path
0,230523_Dataset01_Valeo_Nr01_0.png,dist.1,bad,/content/drive/MyDrive/zerop/FTI Dataset 2023/...
1,230523_Dataset01_Valeo_Nr01_0.png,e.rought1,bad,/content/drive/MyDrive/zerop/FTI Dataset 2023/...
2,230523_Dataset01_Valeo_Nr01_1.png,e.rought2,bad,/content/drive/MyDrive/zerop/FTI Dataset 2023/...
3,230523_Dataset01_Valeo_Nr01_1.png,e.rought3,bad,/content/drive/MyDrive/zerop/FTI Dataset 2023/...
4,230523_Dataset01_Valeo_Nr01_2.png,angle,good,/content/drive/MyDrive/zerop/FTI Dataset 2023/...


In [ ]:
# replacement_dict = {
#     'dist.1': 'distance number one',
#     'e.rought1': 'edge roughness number one',
#     'e.rought2': 'edge roughness number two',
#     'e.rought3': 'edge roughness number three',
#     'angle': 'rhombus angle',
#     'dist.6': 'distance number six',
#     'dots': 'line of dots',
#     'e.rought4': 'edge roughness number four'
# }

# final_filtered_df['Feature'] = final_filtered_df['Feature'].replace(replacement_dict)

In [ ]:
# replacement_dict = {
#     'dist.1': 'distance between parallel lines (type one)',
#     'e.rought1': 'edge roughness (level one)',
#     'e.rought2': 'edge roughness (level two)',
#     'e.rought3': 'edge roughness (level three)',
#     'angle': 'internal angle of rhombus',
#     'dist.6': 'distance between vertical features (type six)',
#     'dots': 'alignment of dot series',
#     'e.rought4': 'edge roughness (level four)'
# }
# df2['Feature'] = df2['Feature'].replace(replacement_dict)

In [4]:
df2.head()

,File,Feature,Label,absolute_path
0,230523_Dataset01_Valeo_Nr01_0.png,dist.1,bad,/content/drive/MyDrive/zerop/FTI Dataset 2023/...
1,230523_Dataset01_Valeo_Nr01_0.png,e.rought1,bad,/content/drive/MyDrive/zerop/FTI Dataset 2023/...
2,230523_Dataset01_Valeo_Nr01_1.png,e.rought2,bad,/content/drive/MyDrive/zerop/FTI Dataset 2023/...
3,230523_Dataset01_Valeo_Nr01_1.png,e.rought3,bad,/content/drive/MyDrive/zerop/FTI Dataset 2023/...
4,230523_Dataset01_Valeo_Nr01_2.png,angle,good,/content/drive/MyDrive/zerop/FTI Dataset 2023/...


In [ ]:
def generate_prompt(feature, label):
    if label == 'good':
        return f"An inkjet-printed image with no visible defect of {feature}."
    else:
        return f"An inkjet-printed image with a visible defect of {feature}."

In [ ]:
def generate_prompt(feature, label):
    if label == 'good':
        return f"An inkjet-printed image showing optimal quality in the {feature}, with no visible defects or irregularities."
    else:
        return f"An inkjet-printed image displaying a noticeable defect in the {feature}."

In [ ]:
def copy_and_create_metadata(df, image_column, label_column, feature_column, target_dir):
    target_dir = Path(target_dir)
    target_dir.mkdir(parents=True, exist_ok=True)

    image_counter = 0
    data = []

    for _, row in df.iterrows():
        image_path = Path(row[image_column])
        label = row[label_column]
        feature = row[feature_column]

        caption = generate_prompt(feature, label)
        # print(caption)
        new_file_name = f"{image_counter:04d}.png"
        # new_file_path = target_dir / new_file_name

        # copyfile(image_path, new_file_path)
        if label == 'good':
            label = 0
        else:
            label = 1

        data.append({
            'file_name': new_file_name,
            'prompts3': caption,
            'label': label
        })

        image_counter += 1


    # Create a DataFrame and save it as metadata.csv
    df_metadata = pd.DataFrame(data)
    df_metadata.to_csv(target_dir / 'metadata_p3.csv', index=False)
    print(f"metadata.csv created with {len(data)} entries.")

In [7]:
def copy_and_create_metadata(df, file, image_column, label_column, feature_column, target_dir):
    target_dir = Path(target_dir)
    target_dir.mkdir(parents=True, exist_ok=True)

    image_counter = 0
    data = []

    for _, row in df.iterrows():
        original_file = Path(row[file])
        image_path = Path(row[image_column])
        label = row[label_column]
        feature = row[feature_column]
        # Map the feature to its specific prompt
        if label == 'good':
            if feature == 'e.rought1':
                caption = "A vertical paralled edges with a slight roughness, where the average deviation from a smooth line. The roughness is consistent and evenly distributed, creating a controlled texture that contrasts with the smooth interior region."
            elif feature == 'dist.1':
                caption = "The horizontal distance between two parallel edges. The gap between these edges is consistently 125.04 pixels across the entire length, indicating a well-defined and defect-free separation."
            elif feature == 'e.rought2':
                caption = "A vertical single edge with pronounced roughness, but the rough texture is irregularly distributed, with certain segments showing unexpected smooth patches, revealing a defect in the edge's consistency."
            elif feature == 'e.rought3':
                caption = "A diagonal edges of the rhombus shape that exhibits roughness, but with uneven and inconsistent textures along its length, suggesting defects in the edge's structure."
            elif feature == 'e.rought4':
                caption = "A diagonal edges of the rhombus shape with a distinct rough texture, characterized by irregularities. The roughness is prominent along the edge, contrasting sharply with smoother adjacent regions."
            elif feature == 'dots':
                caption = "A vertical series of small, uniformly spaced dots aligned in a precise grid pattern. Each dot is consistent in size and spacing, creating a sharply defined grid against the background."
            elif feature == 'angle':
                caption = "The acute angles of a rhombus shape. The angles are clean and well-defined, contributing to the geometric symmetry of the rhombus."
            elif feature == 'dist.6':
                caption = "The distance around one horizontal line, measured consistently across the entire span. This distance is even and uniform, indicating precise alignment and spacing."
        else:
            if feature == 'e.rought1':
                caption = "A vertical edge intended to have a slightly irregular, rough texture, but the roughness is inconsistent, with parts of the edge appearing more jagged and uneven than expected, indicating a defect."
            elif feature == 'dist.1':
                caption = "The horizontal distance between two parallel edges is intended to be 125.04 pixels but displays variation along the length. Gaps range less than 125 pixels, indicating a defect in the spacing."
            elif feature == 'e.rought2':
                caption = "A single vertical edge with pronounced roughness, but the texture is irregularly distributed, with unexpected smooth patches. This inconsistency reveals a defect in the edge's consistency."
            elif feature == 'e.rought3':
                caption = "A diagonal edges of the rhombus shape that exhibits roughness, but the texture is uneven and inconsistent. The roughness varies significantly, suggesting defects in the edge's structure."
            elif feature == 'e.rought4':
                caption = "A diagonal edges of the rhombus shape with intended rough texture, but the roughness is inconsistent. Certain segments appear smoother than intended, indicating a defect along the edge."
            elif feature == 'dots':
                caption = "A vertical series of small aligned in a grid pattern, but the spacing is inconsistent. Some dots are melted or distorted in shape, indicating defects in the grid pattern."
            elif feature == 'angle':
                caption = "An angle formed by two edges that should be 90 degrees but is distorted or uneven. This deviation from the intended geometry suggests a defect in the angle formation."
            elif feature == 'dist.6':
                caption = "The distance around one horizontal line is intended to be 5 pixels but is irregular, with inconsistent spacing along the length, indicating a defect in the alignment."


        new_file_name = f"{image_counter:04d}.png"
        # new_file_path = target_dir / new_file_name

        # copyfile(image_path, new_file_path)

        # label = 1 if label == 'good' else 0
        text = "This image contains more than one feature but focus only on: " + caption
        data.append({
            'file': original_file,
            'file_name': new_file_name,
            'prompts2': text,
            'label': label,
            'feature': feature
        })

        image_counter += 1

    # Create a DataFrame and save it as metadata.csv
    df_metadata = pd.DataFrame(data)
    df_metadata.to_csv(target_dir / 'metadata_p3.csv', index=False)
    print(f"metadata.csv created with {len(data)} entries.")

In [8]:
copy_and_create_metadata(df2, 'File', 'absolute_path', 'Label', 'Feature', '/content/drive/MyDrive/huggingface_format')

metadata.csv created with 960 entries.
